# Workshop 3: Cleaning, Manipulating, and Exploring Data with Pandas
[Placeholder for intro materials]

Notes:
Renaming columns?

In [ ]:
# Import the Pandas library as pd (callable in our code as pd)
import pandas as pd

## Combining datasets
Remove this one?

## Removing data
### Drop columns
We can reduce the size of our dataset by removing unnecessary columns of data using the DataFrame `drop()` method.

In [ ]:
# Remove the "STATE", "FAAREGION", and "COMMENT" columns using "drop()""
wl_strikes_drop = wl_strikes_join.drop(columns=["STATE", "FAAREGION", "COMMENT"])

# Print out the first five records of the DataFrame
wl_strikes_drop.head()

### Remove NaN

## Calculating new data
### Calculating with Expressions
We may want to add a new column that is calculated based on other columns. In this example, we create a new column (`SINGLE_OR_MULTI_ENGINE`) of boolean values that tells us if the plane was a single-engine (TRUE) or a multi-engine (FALSE) plane using a comparison operator to test if the value in the column `NUM_ENGS` equals 1.

In [ ]:
# Create a new column of boolean values indicating single- or multi-engine
wl_strikes_drop['SINGLE_OR_MULTI_ENGINE'] = wl_strikes_drop['NUM_ENGS'] == 1

# Print out the new column
wl_strikes_drop[['NUM_ENGS', 'SINGLE_OR_MULTI_ENGINE']]

### Calculating with apply functions
Sometimes you need to create a new column based on more complex manipulation of existing data. In this example, we use the `apply()` method to apply a function along the rows in the column `TIME` to that parses an integer value of the hour from a string containing the time at which a strike occured. We create a new column `HOUR` that contains a numerical representation of the hour in which a strike occured.

In [ ]:
# Define a function that takes a time string in the form "HH:MM" and returns the
# hour as an integer if the hour value is valid
def calc_hour(time_str):
    hour = time_str.split(':')[0]
    if hour.strip(' ') != '':
        return int(hour)

# Create a new column "HOUR" that contains the hour in which a strike occured
wl_strikes_drop['HOUR'] = wl_strikes_drop['TIME'].apply(calc_hour)

# Print out the new column of data
wl_strikes_drop[['TIME','HOUR']]

## Replacing data
We can replace values in a column based on conditions, similar to "find and replace." In this example, we make our new `SINGLE_OR_MULTI_ENGINE` column more descriptive by changing `True` into " Single engine" and `False` into "Multi engine".

In [ ]:
# Replace True or False values with new strings, "Single engine" or "Multi engine"
wl_strikes_drop['SINGLE_OR_MULTI_ENGINE'].replace({True: 'Single engine', False: 'Multi engine'}, inplace=True)

# Print out the updated column of data
wl_strikes_drop['SINGLE_OR_MULTI_ENGINE']

## Filtering data
### Conditional filtering
We can filter our data using conditional statements. This can help us remove unecessary rows of data or observe a specific range of data.

In [ ]:
# Filter the data to only see incidents that happened at night
night = wl_strikes_drop[wl_strikes_drop['TIME_OF_DAY'] == 'Night']

# Print out the filtered data
night

In [ ]:
# Filter the data to only see data from 2010 and after
data_10s = wl_strikes_drop[wl_strikes_drop['INCIDENT_YEAR'] >= 2010]

# Print out the filtered data
data_10s

In [ ]:
# Filter the data to only see incidents from 2010 and after that happened at night
data_night_10s = wl_strikes_drop[(wl_strikes_drop['INCIDENT_YEAR'] >= 2010) & (wl_strikes_drop['TIME_OF_DAY'] == 'Night')]

# Print out the filtered data
data_night_10s

## Aggregating data
### Unique
It may be interesting to see the total number of unique species that are recorded in our dataset. We can use the `unique()` method on the `SPECIES` column to create an array of unique species names. 

The length of this array will provide the number of unique species.

In [ ]:
# Create a list of the unique species with unique()
unique_species = wl_strikes['SPECIES'].unique()

# Print out the unique species
unique_species

In [ ]:
# Get the length of the new array (How many unique species are there?)
len(unique_species)

### Value counts
Value counts show how many instances there are of each unique entry in a column. Here, we are interested in seeing how many incidents there are in each month. This could tell us which months have the most accidents.

We will specify the `INCIDENT MONTH` column in our dataset and call the method `value_counts()`. This will return a Series with an index label of each unique month number and a value corresponding to the count of that month number in the `INCIDENT MONTH` column.

In [ ]:
# Count the occurance of unique values on the column 'INCIDENT_MONTH'
month_counts = wl_strikes['INCIDENT_MONTH'].value_counts().sort_index()

# Sort the Series by its index using sort_index(), to help view yearly trend
month_counts

### Minimum, maximum, average
We can also calculate aggregates like the minimum, maximum, and mean of values in a DataFrame or Series. Here are a few examples:

- `mean()` to find the average of a range
- `min()` to find the smallest value
- `max()` to find the largest value
- `sum()` to sum the values of a range

In [ ]:
# Calculate the minimum values for each column
wl_strikes.min()

In [ ]:
# Calculate the average height at which all strikes occurred
wl_strikes['HEIGHT'].mean()

We can use the `agg()` method to call multiple aggregate functions at once.

In [ ]:
# Calculate the minimum, maximum, and average hour in which all strikes occurred
wl_strikes['HOUR'].agg(['min', 'max', 'mean'])

## Grouping data
We may be interested in seeing our data in groups. For example, what does the data look like if we group by month and find the average of each column? Which month has the highest cost for repairs? Are there any other values that trend with cost for repairs?

We can do this by calling `groupby()` on our dataset and passing in the column we would like to group by. We will group our data by the column `MONTH`, find the mean of each column in the grouped data, and sort by the `COST_REPAIRS_INFL_ADJ` column to see which month has the highest average cost.

In [ ]:
# Group the dataset by "INCIDENT_MONTH"
by_month = wl_strikes.groupby('INCIDENT_MONTH')

# This creates a groupby object that contains information about the groups
type(by_month)

# Find the mean of the grouped data, then sort by "COST_REPAIRS_INFL_ADJ"
by_month = by_month.mean().sort_values('COST_REPAIRS_INFL_ADJ')

by_month['COST_REPAIRS_INFL_ADJ']

You can also use `groupby()` group data by multiple factors. Here we are grouping by `INCIDENT_MONTH` and then `PRECIPITATION` to see the counts for different types of precipitation in each month using the `size()` groupby method.

In [ ]:
# Group the data by month and then precipitation and get the count of each type
# of precipitation during each month using size()
wl_strikes.groupby(['INCIDENT_MONTH', 'PRECIPITATION']).size()

## Visualization